**Predicting Tennis matches results with Random Forest algorithm**

After understanding the dataset and how the tennis league I will try to predict the results of the matches.

In [1]:
# Import the packages I will need in the developemnt of my machine learning algorithm

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Import the data set I downloaded and uploaded to Kaggle ("Compact tennis dataset")
# After uploading and retrieve the file, I will ask teh program to output it's path, so that I will be able to import it.

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/atp-tennis-2000-2023daily-pull/atp_tennis.csv


**Showing the data for the first time**

After importing the dataSets into my notebook, with the method .head() I will be able to show the first 5 rows of the DataSet, and in addittion, I will be able to take a look at the columns name, so that the develop of the algorithm will be easier.

Since I have imported many .csv files, I also need to specify which file to show.

In [2]:
matches = pd.read_csv("/kaggle/input/atp-tennis-2000-2023daily-pull/atp_tennis.csv")
matches.head()

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score
0,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,Ljubicic I.,Dosedel S.,63,77,-1,-1,-1.0,-1.0,6-4 6-2
1,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Clement A.,Enqvist T.,Enqvist T.,56,5,-1,-1,-1.0,-1.0,3-6 3-6
2,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,Baccanello P.,Escude N.,40,655,-1,-1,-1.0,-1.0,6-7 7-5 6-3
3,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Knippschild J.,Federer R.,Federer R.,87,65,-1,-1,-1.0,-1.0,1-6 4-6
4,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Fromberg R.,Woodbridge T.,Fromberg R.,81,198,-1,-1,-1.0,-1.0,7-6 5-7 6-4


**Understand better the Shape of the Dataset**

After taking a first look at the DataSet we can take a look at the shape of it, which means how many column and rows the file actually have, in this case we start working on the DataSet when the overall shape of it it's going to be 61441 rows an 17 columns

In [3]:
# Shows how many columns and row are in the dataset
matches.shape

(61441, 17)

**Fixing the datatypes**

For most of machine algorithm to work, they must work with integers DataType, and the Random Forest also need. So we have to convert some of the DataType from object into integer, or into some kind of integers. First of all we have to check what DataType we have in our DataSet, then we can choose the Data we want to work with, and if they are not integers already we have to convert it.

In [4]:
# Check what DataType we have in our DataSet
matches["date"] = pd.to_datetime(matches["Date"])
matches.dtypes

Tournament            object
Date                  object
Series                object
Court                 object
Surface               object
Round                 object
Best of                int64
Player_1              object
Player_2              object
Winner                object
Rank_1                 int64
Rank_2                 int64
Pts_1                  int64
Pts_2                  int64
Odd_1                float64
Odd_2                float64
Score                 object
date          datetime64[ns]
dtype: object

**Converting the Date type**

Here I decided to only convert the Date column, but in order to increase the accuracy, in the future more columns may be converted and used in teh algorithm

**Creating the Predictors**

In the Random Forest algorithm, predictors, also known as features or independent variables, are the variables used to make predictions. These predictors are the characteristics or attributes of the data that are input into the algorithm to train the model and subsequently used to make predictions on new, unseen data.

In the context of predicting Premier League football matches, our predictors will include various features related to the result, opponents, time and day of the week. In order to increase the accuracy these predictors might include team statistics (e.g., previous match results, goal difference, number of goals scored), player performance metrics (e.g., goals, assists, yellow cards), historical data (e.g., head-to-head records, past performance in similar scenarios), and various other factors that might influence the outcome of a match.

In [5]:
# Create a new column representing the Indoor or Outdoor value
matches["courtType"] = matches["Court"].astype("category").cat.codes

# Create a new column representing the kind of surface
matches["groundType"] = matches["Surface"].astype("category").cat.codes

# Create a new column representing a code for each opponent
matches["opponentCode"] = matches["Player_2"].astype("category").cat.codes

# Create a new column representing if the player1 won the game with a 1 or if it didn't with a 0
matches["target"] = (matches["Winner"] == matches["Player_1"]).astype("int")













In [6]:
matches

,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,...,Pts_1,Pts_2,Odd_1,Odd_2,Score,date,courtType,groundType,opponentCode,target
0,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,Ljubicic I.,Dosedel S.,...,-1,-1,-1.00,-1.00,6-4 6-2,2000-01-03,1,3,730,1
1,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Clement A.,Enqvist T.,Enqvist T.,...,-1,-1,-1.00,-1.00,3-6 3-6,2000-01-03,1,3,349,0
2,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,Baccanello P.,Escude N.,...,-1,-1,-1.00,-1.00,6-7 7-5 6-3,2000-01-03,1,3,62,1
3,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Knippschild J.,Federer R.,Federer R.,...,-1,-1,-1.00,-1.00,1-6 4-6,2000-01-03,1,3,369,0
4,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Fromberg R.,Woodbridge T.,Fromberg R.,...,-1,-1,-1.00,-1.00,7-6 5-7 6-4,2000-01-03,1,3,1367,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61436,Vienna Open,2023-10-27,ATP500,Indoor,Hard,Quarterfinals,3,Rublev A.,Zverev A.,Rublev A.,...,4765,3460,2.00,1.80,6-1 6-7 6-3,2023-10-27,0,3,1407,1
61437,Vienna Open,2023-10-27,ATP500,Indoor,Hard,Quarterfinals,3,Tiafoe F.,Sinner J.,Sinner J.,...,2400,5000,4.33,1.22,3-6 4-6,2023-10-27,0,3,1162,0
61438,Vienna Open,2023-10-28,ATP500,Indoor,Hard,Semifinals,3,Medvedev D.,Tsitsipas S.,Medvedev D.,...,7355,4300,1.33,3.40,6-4 7-6,2023-10-28,0,3,1267,1
61439,Vienna Open,2023-10-28,ATP500,Indoor,Hard,Semifinals,3,Rublev A.,Sinner J.,Sinner J.,...,4765,5000,2.63,1.50,5-7 6-7,2023-10-28,0,3,1162,0


**Training the model**

As soon as our basic predictors are created and added to the DataSet, we are able to Train our model. Training is one of the most, if not the most important part of our machine learnig algorithm, in my case, I decided to use all the data until the year 2023 as Training Data, and all the Data after the 2023 as testing data.

**Random Forest algorithm**

During the training phase of the Random Forest algorithm, each decision tree is trained to predict the target variable (in this case, the outcome of the Premier League football match) based on the subset of predictors assigned to it. The algorithm determines the importance of each predictor by measuring how much it contributes to reducing the impurity or increasing the information gain in the decision tree.

When making predictions with the trained Random Forest model, the algorithm combines the predictions of all the decision trees by averaging (for regression problems) or voting (for classification problems) to generate the final prediction. Each decision tree contributes its own prediction based on the assigned subset of predictors, and the final prediction is determined by the collective wisdom of the entire ensemble.

In summary, predictors in the Random Forest algorithm are the input variables that capture the relevant information and characteristics of the data used to train the model. These predictors are used by individual decision trees within the ensemble to make predictions, and the algorithm combines the predictions of all the decision trees to produce the final prediction.

In [7]:
# Training the machine learning model 
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 1000, min_samples_split = 10, random_state = 1)

# Divide the dataset into train and test sections
train = matches[matches["date"] < '2023-01-01']
test = matches[matches["date"] > '2023-01-01']

# A list of all the prdictors 
predictors = ["opponentCode", "groundType", "courtType", "Rank_1", "Rank_2", "Pts_1", "Pts_2", "Odd_1", "Odd_2"]

# Fit the random forest model, fitting the predictors and trying to predict the target
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=1000, random_state=1)

**Generating the first prediction**

After succeffully train the model, we are ready to generate some prediction based the previous results. As soon as we have our first prediction we can then calculate the accuracy of the model.

In [8]:
# Generate prediction, passing the test data and the predictors
preds = rf.predict(test[predictors])

# Need to describe the accuracy
from sklearn.metrics import accuracy_score

# Calculate the accuracy passing the test data, with the predictors and the prediction
acc = accuracy_score(test["target"], preds)

**Accuracy**

Our first acccuracy is actually pretty good, but we have to remember that this accuracy isn't just the winning case accuracy, but it's going to reflect also the loose and draw case. We will have to chance the method of calculation for the accuracy

In [9]:
acc

0.6845070422535211

In [10]:
# Create a new DataFrame to combine the actual values and the predicted values 
combined = pd.DataFrame(dict(actual=test["target"], prediction=preds))

# Show a table with the new DataFrame
pd.crosstab(index = combined["actual"], columns = combined["prediction"])

prediction,0,1
actual,,
0,842,400
1,384,859


In [11]:
combined = pd.DataFrame(dict(actual = test["target"], predicted = preds), index = test.index)
combined = combined.merge(matches[["Player_1", "Player_2"]], left_index = True, right_index = True)
combined 

,actual,predicted,Player_1,Player_2
58956,1,1,Kecmanovic M.,O Connell C.
58957,0,1,Rune H.,Nishioka Y.
58958,1,0,Popyrin A.,Auger-Aliassime F.
58959,0,1,Thompson J.,Halys Q.
58960,1,0,Safiullin R.,Ymer M.
...,...,...,...,...
61436,1,0,Rublev A.,Zverev A.
61437,0,0,Tiafoe F.,Sinner J.
61438,1,1,Medvedev D.,Tsitsipas S.
61439,0,1,Rublev A.,Sinner J.
